In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import pymysql
import getpass

In [3]:
# Let's connect to our database
conn = pymysql.connect(host="35.233.174.193",port=3306,
                       user="jovyan",passwd=getpass.getpass("Enter password for MIMIC2 database"),
                       db='mimic2')

Enter password for MIMIC2 database········


In [4]:
df = pd.read_sql("""SELECT * FROM noteevents WHERE category = "DISCHARGE_SUMMARY"  LIMIT 10;""", conn)

In [5]:
df.head()

,subject_id,hadm_id,icustay_id,elemid,charttime,realtime,cgid,correction,cuid,category,title,text,exam_name,patient_info
0,56,28766,None,None,2644-01-17 00:00:00,None,None,None,None,DISCHARGE_SUMMARY,None,\n \n \n \nAdmission Date: [**2644-1-17**] ...,None,None
1,37,18052,None,None,3264-08-14 00:00:00,None,None,None,None,DISCHARGE_SUMMARY,None,\n\n\nAdmission Date: [**3264-8-14**] Dis...,None,None
2,78,15161,None,None,2778-03-24 00:00:00,None,None,None,None,DISCHARGE_SUMMARY,None,\n\nAdmission Date: [**2778-3-24**] Dis...,None,None
3,26,15067,None,None,3079-03-03 00:00:00,None,None,None,None,DISCHARGE_SUMMARY,None,\n\nAdmission Date: [**3079-3-3**] Discha...,None,None
4,12,12532,None,None,2875-09-26 00:00:00,None,None,None,None,DISCHARGE_SUMMARY,None,\n\n\nAdmission Date: [**2875-9-26**] Dis...,None,None


In [6]:
texts = df["text"]

In [7]:
import json

In [213]:
with open("../resources/section_patterns.json") as f:
    patterns = json.load(f)

In [214]:
len(patterns)

14

In [215]:
import re
compiled_patterns = []
for name, sect_patterns in patterns.items():
    compiled_patterns.append({"section": name, 
                              "patterns": [re.compile(pattern, re.IGNORECASE) for pattern in sect_patterns]})

In [216]:
class Sectionizer:
    def __init__(self):
        self.sections = set()
        self.patterns = {}
    
    def add(self, patterns):
        for sect_patterns in patterns:
            name = sect_patterns["section"]
            self.sections.add(name)
            self.patterns.setdefault(name, list())
            self.patterns[name] += sect_patterns["patterns"]
            
    def extract_sections(self, text):
        matches = []
        for name, sect_patterns in self.patterns.items():
            for pattern in sect_patterns:
                sect_matches = list(pattern.finditer(text))
                for match in sect_matches:
                    matches.append((name, match))
        if len(matches) == 0:
            return [(None, text)]
        
        matches = sorted(matches, key=lambda x: (x[1].start(), 0-x[1].end()))
        matches = self._dedup_matches(matches)
        
        
        
        sections = []
        if matches[0][1].start() != 0:
            sections.append(("UNK", text[:matches[0][1].start()]))
        for i, (name, match) in enumerate(matches):
            if i == len(matches) - 1:
                sections.append((name, text[match.start():]))
            else:
                next_match = matches[i+1][1]
                sections.append((name, text[match.start():next_match.start()]))
            
            
        
        return sections

    def _dedup_matches(self, matches):
        deduped = []
        # TODO: Make this smarter
        deduped.append(matches[0])
        for i, match in enumerate(matches[1:], start=1):
            if not self._overlaps(deduped[-1], match):
                deduped.append(match)
        return deduped
            
    def _overlaps(self, a, b):
        (_, a) = a
        (_, b) = b
        if a.start() <= b.start() < a.end():
            return True
        if b.start() <= a.start() < b.end():
            return True

In [217]:
sectionizer = Sectionizer()
sectionizer.add(compiled_patterns)

In [218]:
matches = sectionizer.extract_sections(texts[1])

In [219]:
for (sect, sect_text) in matches:
    print(sect)
    print(sect_text)
    print("----"*10)
    print()

UNK



Admission Date:  [**3264-8-14**]     Discharge Date:  [**3264-8-19**]

Date of Birth:   [**3195-9-11**]     Sex:  M

Service:  C-MED Gold


----------------------------------------

Present Illness
HISTORY OF PRESENT ILLNESS:  The patient is a 68-year-old
gentleman with a significant cardiac history with coronary
artery disease (status post coronary artery bypass graft),
hypertension, congestive heart failure, hyperlipidemia, and
diabetes mellitus who presented with worsening congestive
heart failure of several months in duration with new onset
chest pressure.

The patient was in his usual state of health until three days
prior to admission when he began to experience increasing
shortness of breath at rest and with exertion. However, the
patient was still able to walk his dog for approximately one
half of a mile per day.  No paroxysmal nocturnal dyspnea.  No
orthopnea.  Minimal bilateral lower extremity edema at
baseline.

The patient had one episode of 5/10 chest pressure (his


In [221]:
compiled_patterns[0]

{'section': 'pmh',
 'patterns': [re.compile(r'(past )?medica\s+(history|hx):',
  re.IGNORECASE|re.UNICODE),
  re.compile(r'\bmhx?:', re.IGNORECASE|re.UNICODE),
  re.compile(r'\bmh?:', re.IGNORECASE|re.UNICODE),
  re.compile(r'pohx:', re.IGNORECASE|re.UNICODE),
  re.compile(r'pmh?:', re.IGNORECASE|re.UNICODE),
  re.compile(r'past\s*history:', re.IGNORECASE|re.UNICODE),
  re.compile(r'history:', re.IGNORECASE|re.UNICODE),
  re.compile(r'p[hm]+\s*computerized\s+problem\s+list:',
  re.IGNORECASE|re.UNICODE),
  re.compile(r'significant\smedical\shx:', re.IGNORECASE|re.UNICODE),
  re.compile(r'past\smedical\shistory\s?\/\s?problem list:',
  re.IGNORECASE|re.UNICODE),
  re.compile(r'Past surgical history:', re.IGNORECASE|re.UNICODE),
  re.compile(r'patient_history:', re.IGNORECASE|re.UNICODE),
  re.compile(r'patient history:', re.IGNORECASE|re.UNICODE),
  re.compile(r'pt history:', re.IGNORECASE|re.UNICODE),
  re.compile(r'history/physical examination:', re.IGNORECASE|re.UNICODE),
  re.compil

In [222]:
section_strings = []
for text in texts:
    for pattern_dict in compiled_patterns:
        name = pattern_dict["section"]
        sect_patterns = pattern_dict["patterns"]
        
        for pattern in sect_patterns:
            search = pattern.search(text)
            if search:
                print(name, search)
                section_strings.append((name, search.group()))

pmh <re.Match object; span=(900, 908), match='History:'>
pmh <re.Match object; span=(900, 908), match='History:'>
pmh <re.Match object; span=(887, 908), match='Past Medical History:'>
pmh <re.Match object; span=(892, 908), match='Medical History:'>
sexual and social history <re.Match object; span=(980, 995), match='Social History:'>
Observation_and_plan <re.Match object; span=(4849, 4859), match='Diagnosis:'>
Observation_and_plan <re.Match object; span=(4849, 4859), match='Diagnosis:'>
medication <re.Match object; span=(4714, 4736), match='Discharge Medications:'>
medication <re.Match object; span=(4724, 4736), match='Medications:'>
medication <re.Match object; span=(4645, 4670), match='Medications on Admission:'>
medication <re.Match object; span=(4724, 4736), match='Medications:'>
allergy <re.Match object; span=(164, 174), match='Allergies:'>
allergy <re.Match object; span=(164, 174), match='Allergies:'>
Chief complaint <re.Match object; span=(272, 288), match='Chief Complaint:'>
Phy

In [223]:
section_strings = list(set(section_strings))

In [224]:
section_strings

[('pmh', 'PAST SURGICAL HISTORY:'),
 ('Observation_and_plan', 'DIAGNOSES:'),
 ('medication', 'MEDICATIONS PRIOR TO ADMISSION:'),
 ('Observation_and_plan', 'ASSESSMENT AND PLAN:'),
 ('Present Illness', 'PRESENT ILLNESS:'),
 ('medication', 'MEDICATIONS:'),
 ('family_history', 'Family History:'),
 ('sexual and social history', 'sh:'),
 ('medication', 'Medications on Admission:'),
 ('medication', 'MEDICATIONS ON ADMISSION:'),
 ('Present Illness', 'HISTORY OF PRESENT ILLNESS:'),
 ('Observation_and_plan', 'Diagnosis:'),
 ('sexual and social history', 'Social History:'),
 ('pmh', 'MEDICAL HISTORY:'),
 ('pmh', 'PAST MEDICAL HISTORY:'),
 ('pmh', 'Medical History:'),
 ('family_history', 'FAMILY HISTORY:'),
 ('medication', 'Medications:'),
 ('medication', 'MEDICATIONS ON DISCHARGE:'),
 ('sexual and social history', 'SOCIAL HISTORY:'),
 ('pmh', 'pm:'),
 ('pmh', 'History:'),
 ('Chief complaint', 'CHIEF COMPLAINT:'),
 ('medication', 'Discharge Medications:'),
 ('allergy', 'Allergies:'),
 ('pmh', 'HI

# Version 2: SpaCy

In [225]:
from spacy.matcher import Matcher, PhraseMatcher

In [226]:
def prune_overlapping_matches(matches, strategy="longest"):
    if strategy != "longest":
        raise NotImplementedError()

    # Make a copy and sort
    unpruned = sorted(matches, key=lambda x: (x[1], x[2]))
    pruned = []
    num_matches = len(matches)
    if num_matches == 0:
        return matches
    curr_match = unpruned.pop(0)

    while True:
        if len(unpruned) == 0:
            pruned.append(curr_match)
            break
        next_match = unpruned.pop(0)

        # Check if they overlap
        if overlaps(curr_match, next_match):
            # Choose the larger span
            longer_span = max(curr_match, next_match, key=lambda x: (x[2] - x[1]))
            pruned.append(longer_span)
            if len(unpruned) == 0:
                break
            curr_match = unpruned.pop(0)
        else:
            pruned.append(curr_match)
            curr_match = next_match
    # Recursive base point
    if len(pruned) == num_matches:
        return pruned
    # Recursive function call
    else:
        return prune_overlapping_matches(pruned)

def overlaps(a, b):
    if _span_overlaps(a, b) or _span_overlaps(b, a):
        return True
    return False

def _span_overlaps(a, b):
    _, a_start, a_end = a
    _, b_start, b_end = b
    if a_start >= b_start and a_start < b_end:
        return True
    if a_end > b_start and a_end <= b_end:
        return True
    return False

def matches_to_spans(doc, matches, set_label=True):
    spans = []
    for (rule_id, start, end) in matches:
        if set_label:
            label = doc.vocab.strings[rule_id]
        else:
            label = None
        spans.append(Span(doc, start=start, end=end, label=label))
    return spans

In [227]:
import spacy

In [133]:
nlp = spacy.load("en_core_web_sm", disable="ner")

In [293]:
class DocSectionizer:
    name = "sectionizer"
    def __init__(self, nlp):
        self.nlp = nlp
        self.matcher = Matcher(nlp.vocab)
        self.phrase_matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
        
        self.rules = []
        
    def add(self, patterns):
        for pattern_dict in patterns:
            name = pattern_dict["name"]
            pattern = pattern_dict["pattern"]
            if isinstance(pattern, str):
                self.phrase_matcher.add(name, None, self.nlp.make_doc(pattern))
            else:
                self.matcher.add(name, pattern)

    def __call__(self, doc):
        matches = self.matcher(doc)
        
        matches += self.phrase_matcher(doc)
        
        matches = prune_overlapping_matches(matches)
        if len(matches) == 0:
            return doc
        
        first_match = matches[0]
        section_spans = []
        if first_match[1] != 0:
            section_spans.append((None, None, doc[0:first_match[1]]))
        for i, match in enumerate(matches):
            (match_id, start, end) = match
            section_header = doc[start:end]
            name = self.nlp.vocab.strings[match_id]
            if i == len(matches) - 1:
                section_spans.append((name, section_header, doc[start:]))
            else:
                next_match = matches[i+1]
                _, next_start, _ = next_match
                section_spans.append((name, section_header, doc[start:next_start]))
        for name, header, section in section_spans:
            doc._.sections.append((name, header, section))
            for token in section:
                token._.section = section
                token._.section_name = name
                token._.section_header = header
        return doc

In [259]:
from spacy.tokens import Doc, Span, Token

In [260]:
def get_sections(doc):
    sections = []
    for (section_name, (start, end)) in doc._.section_spans:
        sections.append((section_name, doc[start:end]))
    return sections

In [261]:
Doc.set_extension("sections", default=list(), force=True)

Token.set_extension("section", default=None, force=True)
Token.set_extension("section_name", default=None, force=True)
Token.set_extension("section_header", default=None, force=True)

Span.set_extension("section", getter=lambda x: x[0]._.section, force=True)
Span.set_extension("section_name", getter=lambda x: x[0]._.section_name, force=True)
Span.set_extension("section_header", getter=lambda x: x[0]._.section_header, force=True)

In [262]:
doc = nlp(text)

In [263]:
section_spacy_patterns = [{"name": name, "pattern": pattern}
                         for (name, pattern) in section_strings]

In [264]:
sectionizer = DocSectionizer(nlp)

In [265]:
sectionizer.add(section_spacy_patterns)

In [266]:
doc = sectionizer(doc)

In [269]:
def get_section(section_name, doc):
    return [(name, header, section) for (name, header, section)
           in doc._.sections if name == section_name]
        

In [272]:
get_section("pmh", doc)

[('pmh', Past Medical History:, Past Medical History:),
 ('pmh', PAST MEDICAL HISTORY:, PAST MEDICAL HISTORY:  
  1. ESRD secondary to hypertensive nephrosclerosis s/p right 
  upper extremity AV graft 9'[**61**]'[**38**] in preparation for dialysis. 
  Graft placement was complicated by cellulitis, for which he was 
  treated with keflex  
  2. DM, on glyburide and glipizide at home  
  3. HTN, on clonidine, lisinopril, nifedipine  
  4. PVD s/p aortic bypass  
  5. CVA, with residual weakness of his left side  
  6. R CEA  
  7. Secondary hyperparathyroidism  
  8. Chronic anemia on procrit injections  
  9. Prostate CA on Lupron  
  10. Gout  
  
   ),
 ('pmh', pm:, pm: read pending  
  .
  [**3138-10-30**] Cath
  COMMENTS:      
  1. Right heart catheterization revealed elevated right and left 
  sided 
  pressures. (PCWP = 25 mmhg). 
  2. Left heart catheterization revealed no evidence of systolic 
  hypertension.  Calculated cardiac output and index were 5.0/2.8. 
  
  3. Selecti

# Demo

In [300]:
text = """
Family History:
Diabetes

Past Medical History:
Pneumonia

Assessment and Plan:
Atrial fibrillation. There is no evidence of pneumonia.
"""

In [275]:
import os, sys

In [276]:
sys.path.append("../../nlp_tools/")

In [277]:
import nlp_tools

In [283]:
nlp = nlp_tools.build_nlp()

In [280]:
nlp.pipe_names

['tagger', 'parser', 'target_matcher', 'context', 'postprocessor']

In [285]:
target_matcher = nlp.get_pipe('target_matcher')

In [287]:
target_rules = [
    nlp_tools.TargetRule("diabetes", "PROBLEM"),
    nlp_tools.TargetRule("pneumonia", "PROBLEM"),
    nlp_tools.TargetRule("atrial fibrillation", "PROBLEM"),
]

In [288]:
target_rules

In [289]:
target_matcher.add(target_rules)

In [294]:
sectionizer = DocSectionizer(nlp)

In [295]:
section_rules = [
    {"name": "family_history", "pattern": "Family History:"},
    {"name": "past_medical_history", "pattern": "Past Medical History:"},
    {"name": "assessment_and_plan", "pattern": "Assessment and Plan:"},
]

In [296]:
sectionizer.add(section_rules)

In [297]:
nlp.add_pipe(sectionizer, after="parser")

In [298]:
nlp.pipe_names

['tagger',
 'parser',
 'sectionizer',
 'ner',
 'target_matcher',
 'context',
 'postprocessor']

In [301]:
postprocessor = nlp.get_pipe("postprocessor")

In [306]:
def set_is_family(ent, i, value=True):
    ent._.is_family = value
    
def set_is_historical(ent, i, value=True):
    ent._.is_historical = value

In [303]:
import cycontext

In [305]:
cycontext.context_component.DEFAULT_ATTRS

{'NEGATED_EXISTENCE': {'is_negated': True},
 'POSSIBLE_EXISTENCE': {'is_uncertain': True},
 'HISTORICAL': {'is_historical': True},
 'HYPOTHETICAL': {'is_hypothetical': True},
 'FAMILY': {'is_family': True}}

In [307]:
postprocess_rules = [
    nlp_tools.PostprocessingRule(
        patterns=[
            nlp_tools.PostprocessingPattern(lambda ent: ent._.section_name == "family_history"),
        ],
        action=set_is_family
    ),
    nlp_tools.PostprocessingRule(
        patterns=[
            nlp_tools.PostprocessingPattern(lambda ent: ent._.section_name == "past_medical_history"),
        ],
        action=set_is_historical
    ),
]

In [308]:
postprocessor.add(postprocess_rules)

In [309]:
doc = nlp(text)

In [313]:
doc.ents

(Diabetes, Pneumonia, Atrial fibrillation, pneumonia)

In [318]:
list(nlp("Past Medical History:"))

[Past, Medical, History, :]

In [317]:
for (section_name, section_header, section) in doc._.sections:
    print(section_name, section_header, section, sep="\n")
    print("---"*5)

None
None


---------------
family_history
Family History:
Family History:
Diabetes


---------------
past_medical_history
Past Medical History:
Past Medical History:
Pneumonia


---------------
assessment_and_plan
Assessment and Plan:
Assessment and Plan:
Atrial fibrillation. There is no evidence of pneumonia.

---------------


In [312]:
for ent in doc.ents:
    print(ent, ent._.is_negated, ent._.is_historical, ent._.is_family)

Diabetes False False True
Pneumonia False True False
Atrial fibrillation False False False
pneumonia True False False


In [314]:
for ent in doc.ents:
    print(ent, ent._.section_name)

Diabetes family_history
Pneumonia past_medical_history
Atrial fibrillation assessment_and_plan
pneumonia assessment_and_plan
